# 08Tools: Prediction - Custom

# <IN ACTIVE DEVELOPMENT - NOT COMPLETE>

---
## Setup

### Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (Artifact Registry and Cloud Build)

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These installs must be completed before continuing this notebook.

In [75]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q

### Environment

inputs:

In [76]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [77]:
REGION = 'us-central1'
EXPERIMENT = '08_predictions'
SERIES = '08'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [78]:
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import service_usage_v1
from google.cloud import storage

import requests
import json
import numpy as np

import multiprocessing

clients:

In [79]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()
gcs = storage.Client()
su_client = service_usage_v1.ServiceUsageClient()

parameters:

In [80]:
BUCKET = PROJECT_ID
DIR = f"temp/{EXPERIMENT}"

environment:

In [81]:
!rm -rf {DIR}
!mkdir -p {DIR}

### Enable APIs

Using Cloud Run requires enabling the API for the Google Cloud Project.

Options for enabeling.  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### Cloud Run

In [82]:
cloudrun = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/run.googleapis.com'
    )
).state.name


if cloudrun == 'DISABLED':
    print(f'Cloud Run is currently {cloudrun} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/run.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Run is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Run already enabled for project: {PROJECT_ID}')

Cloud Run already enabled for project: statmike-mlops-349915


---
## Get a Model For Predictions
This project already has a model serving online predictions at a Vertex AI Endpoint.  This section will use the endpoint to retrieve the deployed model and get its information to use for batch prediction methods in this notebook.

### Get Endpoint

[Endpoint Properties and Methods](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint):

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [9]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
endpoint = endpoints[0]

In [10]:
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/1178329019301494784?project=statmike-mlops-349915


### Get Model at Endpoint
Using the model on the endpoint for the current series:

In [11]:
endpoint

resource name: projects/1026793852137/locations/us-central1/endpoints/1178329019301494784

In [12]:
#endpoint.list_models()[0]

In [13]:
model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model+f'@{endpoint.list_models()[0].model_version_id}'
)

### Review Model Information

In [14]:
model.display_name

'08_08f'

In [15]:
model.resource_name

'projects/1026793852137/locations/us-central1/models/model_08_08f'

In [16]:
model.version_id

'3'

In [17]:
model.version_description

'run-20221025152217'

In [18]:
model.versioned_resource_name

'projects/1026793852137/locations/us-central1/models/model_08_08f@3'

In [19]:
model.supported_input_storage_formats

['jsonl', 'bigquery', 'csv', 'tf-record', 'tf-record-gzip', 'file-list']

In [20]:
model.name

'model_08_08f'

In [21]:
model.uri

'gs://statmike-mlops-349915/08/08f/models/20221025152217/model'

In [33]:
model.container_spec

image_uri: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08f"
command: "Rscript"
command: "./code/serve.R"

In [23]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}/versions/{model.version_id}/properties?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_08_08f/versions/3/properties?project=statmike-mlops-349915


#### Review Model Information Using the `aiplatform_v1` Model Client
It may also be helpful to try the [ModelServiceClient](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform_v1.services.model_service.ModelServiceClient) in version 1 of the client to review the model attributes.  Here is example code for trying this.

Curious about client versions and layers?  Check out this tip document [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [24]:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
ModelClientv1 = aiplatform_v1.ModelServiceClient(client_options = client_options)

ModelClientv1.get_model(
    name = model.versioned_resource_name
)

name: "projects/1026793852137/locations/us-central1/models/model_08_08f@3"
display_name: "08_08f"
predict_schemata {
}
metadata {
}
training_pipeline: "projects/1026793852137/locations/us-central1/trainingPipelines/556316324893032448"
container_spec {
  image_uri: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08f"
  command: "Rscript"
  command: "./code/serve.R"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1665961709
  nanos: 474378000
}
update_time {
  seconds: 1666712027
  nanos: 890201000
}
deployed_models {
  endpoint: "projects/1026793852137/locations/us-central1/endpoint

---
## Retrieve Records For Prediction

In [25]:
n = 1000
pred = bq.query(query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE splits='TEST' LIMIT {n}").to_dataframe()

In [26]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0,0,814c62c8-ade4-47d5-bf83-313b0aafdee5,TEST
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0,0,d08a1bfa-85c5-4f1b-9537-1c5a93e6afd0,TEST
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0,0,802f3307-8e5a-4475-b795-5d5d8d7d0120,TEST


Remove columns not included as features in the model:

In [27]:
newobs = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')
#newobs[0]

In [29]:
len(newobs)

1000

In [31]:
newobs[0]

{'Time': 35337,
 'V1': 1.0928441854981998,
 'V2': -0.0132303486713432,
 'V3': 1.35982868199426,
 'V4': 2.7315370965921004,
 'V5': -0.707357349219652,
 'V6': 0.8738370029866129,
 'V7': -0.7961301510622031,
 'V8': 0.437706509544851,
 'V9': 0.39676985012996396,
 'V10': 0.587438102569443,
 'V11': -0.14979756231827498,
 'V12': 0.29514781622888103,
 'V13': -1.30382621882143,
 'V14': -0.31782283120234495,
 'V15': -2.03673231037199,
 'V16': 0.376090905274179,
 'V17': -0.30040350116459497,
 'V18': 0.433799615590844,
 'V19': -0.145082264348681,
 'V20': -0.240427548108996,
 'V21': 0.0376030733329398,
 'V22': 0.38002620963091405,
 'V23': -0.16764742731151097,
 'V24': 0.0275573495476881,
 'V25': 0.59211469704354,
 'V26': 0.219695164116351,
 'V27': 0.0369695108704894,
 'V28': 0.010984441006191,
 'Amount': 0.0}

---

In [34]:
model.container_spec

image_uri: "us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08f"
command: "Rscript"
command: "./code/serve.R"

In [41]:
servercommand = ' '.join(model.container_spec.command)
servercommand

'Rscript ./code/serve.R'

In [36]:
model.container_spec.image_uri

'us-central1-docker.pkg.dev/statmike-mlops-349915/statmike-mlops-349915/08f'

---
## Cloud Run: R Model Server

Cloud Run will be used to host our custom container and handle prediction requests.

### Deploy as Cloud Run Service
This demonstration creates an open service allowing all traffic.  Review documentation for [Cloud Run](https://cloud.google.com/run/docs/overview/what-is-cloud-run) and the [CLOUD SKD CLI sections](https://cloud.google.com/sdk/gcloud/reference/run) for `gcloud run`.


If you have a policy inforced for 'Domain Restricted Sharing' then it may need adjusting for the project to allow this.  This should be done with care and you may wish to only accept authenticated or internal traffic.  Review options for authentication [here](https://cloud.google.com/run/docs/authenticating/overview).

Updated Org Policy:
- Logged in as Admin
- IAM > Organization Policies
    - Changed to Project (not org level)
    - Filter 'Domain Restricted Sharing'
    - Select and Edit
        - Applies to = Customize
        - Policy enforcement = Replace
        - Rules = Allow all
    - Save

View the Cloud Run Console for this project:

In [38]:
print(f'https://console.cloud.google.com/run?project={PROJECT_ID}')

https://console.cloud.google.com/run?project=statmike-mlops-349915


Create the Cloud Run Service:

In [110]:
model.container_spec.command

['Rscript', './code/serve.R']

In [111]:
!gcloud run deploy endpoint-$SERIES --image=$model.container_spec.image_uri --command='Rscript' --command='./code/serve_cloudrun.R' --port=8080 --region=$REGION --platform=managed --allow-unauthenticated --no-user-output-enabled

Deploying new service...                                                       
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      


In [112]:
!gcloud run services list

   SERVICE      REGION       URL                                          LAST DEPLOYED BY                                     LAST DEPLOYED AT
✔  endpoint-08  us-central1  https://endpoint-08-urlxi72dpa-uc.a.run.app  1026793852137-compute@developer.gserviceaccount.com  2022-10-28T20:05:35.030537Z


In [113]:
services = !gcloud run services list --format="json" --filter=SERVICE:endpoint-$SERIES
services = json.loads("".join(services))[0]
services['status']['url']

'https://endpoint-08-urlxi72dpa-uc.a.run.app'

If you had to adjust a `Domain Restricted Sharing` policy after deployment then this command can update the service to allow all traffic:

In [92]:
#!gcloud run services add-iam-policy-binding --region=us-central1 --member='allUsers' --role=roles/run.invoker endpoint-$SERIES

### Get Predictions Using Cloud Run Service

Prepare in instance (observation) for prediction request:

In [129]:
tryob_json = json.dumps({"instances": newobs[0:2]})
tryob_json

'{"instances": [{"Time": 35337, "V1": 1.0928441854981998, "V2": -0.0132303486713432, "V3": 1.35982868199426, "V4": 2.7315370965921004, "V5": -0.707357349219652, "V6": 0.8738370029866129, "V7": -0.7961301510622031, "V8": 0.437706509544851, "V9": 0.39676985012996396, "V10": 0.587438102569443, "V11": -0.14979756231827498, "V12": 0.29514781622888103, "V13": -1.30382621882143, "V14": -0.31782283120234495, "V15": -2.03673231037199, "V16": 0.376090905274179, "V17": -0.30040350116459497, "V18": 0.433799615590844, "V19": -0.145082264348681, "V20": -0.240427548108996, "V21": 0.0376030733329398, "V22": 0.38002620963091405, "V23": -0.16764742731151097, "V24": 0.0275573495476881, "V25": 0.59211469704354, "V26": 0.219695164116351, "V27": 0.0369695108704894, "V28": 0.010984441006191, "Amount": 0.0}, {"Time": 60481, "V1": 1.23897317836686, "V2": 0.0352260291957428, "V3": 0.0630025820449367, "V4": 0.641405616773756, "V5": -0.260893052049767, "V6": -0.580097475501787, "V7": 0.0499378052604491, "V8": -0.

Make prediction request:

In [130]:
json_response = requests.post(f"{services['status']['url']}/predict", data=tryob_json, headers={"content-type": "application/json"})

In [131]:
json_response

<Response [200]>

In [132]:
print(json_response.text)

{"predictions":[0.0018,0.0004]}


In [133]:
predictions = json.loads(json_response.text)['predictions']
predictions

[0.0018, 0.0004]

In [134]:
np.argmax(predictions[0])

0

### Remove Service
This command will remove the service.

Alternatively, you could adjust the service to not accept traffic.  Cloud Run will scale down to zero - or only charge when CPU is used (startup, shutdown, and receiving requests) unless `--no-cpu-throttling` is used ([documentation](https://cloud.google.com/run/docs/configuring/cpu-allocation#setting)).

In [135]:
!gcloud run services delete --region=us-central1 --quiet endpoint-$SERIES

Deleting [endpoint-08]...done.                                                 
Deleted service [endpoint-08].
